In [69]:
import numpy as np
import pandas as pd
#import asyncio
#import aiohttp
import json
import csv
import requests
import time
from cryptodata import get_markets, get_pairs, get_ohlc

In [70]:
pd.set_option('display.max_rows',10)

In [3]:
def list_markets_containing_pair(pair='btcusd'):
    try:
        r = requests.get("https://api.cryptowat.ch/markets").json()['result']
        exchanges = {item['exchange'] for item in r if (item['active'] is True and item['pair'] == pair)}
        return exchanges
    except:
        print('error')

def list_pairs(pairs=get_pairs(), p1='btc'):
    pairs = list(pairs)
    L = []
    for pair in pairs:
        if p1 == pair[:len(p1)]:
            L.append(pair)
    return L

def list_pairs_in_market(market):
    r = requests.get("https://api.cryptowat.ch/markets/"+market).json()['result']
    market_pairs = {item['pair'] for item in r if item['active'] is True}
    return market_pairs

In [67]:
def get_ohlc(market, pair, period = 1440, api = "cryptowatch", start = "", end = "", local_timezone = True):
    """get OHLC prices available in API"""
    
    period *= 60
    period = str(period)
    params = []
    dates = ["",""]
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
    
    for i, date in enumerate([start, end]):
        if (not date):
            dates[i] = date
        else:
            try:
                dates[i] = int(time.mktime(time.strptime(date, "%d %m %y %H %M"))) #+ local_timezone*time.timezone
                print(local_timezone,time.timezone,date,dates[i])
                print(time.strptime(date, "%d %m %y %H %M"), time.mktime(time.strptime(date, "%d %m %y %H %M")))
            except:
                print("Error: Datetime format d m y H M")
    #dates[0] = int(1518134400)
    prms = {'before': dates[1], 'after': dates[0]}
    print(dates[0] - local_timezone*time.timezone)
#    print('params',prms)
    
    if api == "cryptowatch":
        try:
            r = requests.get(url, params=prms).json()['result'][period]
            print(r)
            #ohlc = [item['pair'] for item in r]          
            a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
            a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - local_timezone*time.timezone, unit = 's')
            return a        
        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [5]:
def compare_pair_across_markets(pair = 'btcusd'):
    a = list_markets_containing_pair(pair)
    print(a)
    c = {}
    for m in list(a):
        print(m, pair)
        c[m] = get_ohlc(m,pair)
    return c

In [6]:
import os
import shutil

class Pair:
    markets = {}
    data = {}
    def __init__(self, name, start=0, end=time.time(), period=1440):
        self.name = str(name)
        self.period = period    
        self.start = start
        self.end = int(end)
        self.markets = list_markets_containing_pair(self.name)
        self.metadata = pd.DataFrame([period, start, end])
        
    

def download_pair_data(pair, force_update = False):
    try:
        os.mkdir(pair.name)
    except:
        try:
            m = pd.read_csv('pair_metadata.csv')
        except:
#        if not (m == pair.metadata).all():
            shutil.rmtree(pair.name)
            os.mkdir(pair.name)
        #print('m\n',m)
        #print('pair.metadata', pair.metadata)
        #if not (m == pair.metadata).all():
        #    shutil.rmtree(pair.name)
        #    os.mkdir(pair.name)
    
    #print(pair.markets)
    for key in list(pair.markets):
        pair.data[key].to_csv(pair.name+'/'+key+'.csv')
    
    pair.metadata.to_csv('pair_metadata.csv')

def load_pair_data(name, start = 0, end = time.time(), period = 1440):
    mkt = os.listdir(name)
    for i in range(len(mkt)):
        mkt[i] = mkt[i][0:-4]
    return mkt

In [62]:
def Timestamp_from_string(s='31 12 99'):
    #print(time.mktime(time.strptime(s, "%d %m %y")))
    return pd.Timestamp(int(time.mktime(time.strptime(s, "%d %m %y"))), unit='s')